In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import transformers
import glob
import csv
import re

import sys
import pprint
import pandas as pd
import numpy as np
from scipy.special import softmax
import scipy.stats
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import auc, precision_recall_curve, roc_curve, RocCurveDisplay
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score


from sklearn.model_selection import ParameterGrid
import sklearn
import os
import shutil
from os import listdir
from os.path import isfile, join
import time
import torch

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from datasets import load_dataset

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('max_colwidth', 300)
pd.set_option('display.max_columns', 100)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# Setting path for importing required functions for data processing

sys.path.append("data_processing")
sys.path.append("data_processing")

# Functions required for data processing

import process_text
import transform_textfiles

# Functions

### Data processing

In [ ]:
def extractOriginalText2(input_path):
    # read in progress note from txt file (same txt file that was imported to CLAMP for annotation)
    results = glob.glob(f"{input_path}/*.txt")
    rows = []
    
    # extract original note text from the progress note and save as dataframe
    for i in range(len(results)):
        row = pd.read_csv(results[i], sep="\t", quoting=csv.QUOTE_NONE).iloc[:, [6]]
        row.columns = ['note_des']
        row['file'] = os.path.basename(results[i]).split(".")[0]
        rows.append(row)
    return pd.concat(rows)

def extractXMIAnnotation2(input_path):
    # write Patterns
    id_match = re.compile("(?<=xmi:id=\")\d{,5}(?=\")")
    begin_match = re.compile("(?<=begin=\")\d{,6}(?=\")")
    end_match = re.compile("(?<=end=\")\d{,6}(?=\")")
    tag_match = re.compile("(?<=semanticTag=\")\w*(?=\")")

    # read annotation file (notes were annotated in CLAMP)
    # annotation files were exported from CLAMP in XMI format
    results = glob.glob(f"{input_path}/*.xmi")
    rows = []
    
    # extract annotations from XMI files (noted with semanticTag field)
    for i in range(len(results)):
        file = open(results[i], "r+")
        lines = file.readlines()
        lines = lines[0].split('><')
        extract = [x for x in lines if re.search("semanticTag", x)]
        extract = [(id_match.findall(x),
                    begin_match.findall(x),
                    end_match.findall(x),
                    tag_match.findall(x))
                   for x in extract]
        unique_tags = sorted(list(set([x[3][0] for x in extract])))
#         file_name = re.findall("\\A.*(?=\.)",os.path.basename(results[i]))[0].split("-")
        file_name = os.path.basename(results[i]).split(".")[0]
        
        row = pd.DataFrame({'xmi': [os.path.basename(results[i])],
                            'file': [os.path.basename(results[i]).split(".")[0]],
                            'anon_id': [file_name.split("-")[1]],
                            'encounter_id': [file_name.split("-")[2]]})

        for tag in unique_tags:
            row.loc[:, tag] = 1

        rows.append(row)

    full = pd.concat(rows).fillna(0)

    # categorize and process types of ground truth annotations
    ## PTBM: parent training in behavioral management
    ## weak_bt: weak evidence of PTBM
    ## strong_bt: strong evidence of PTBM
    ## bt_yn: binary variable for any evidence of PTBM
    full['weak_bt'] = np.where((full['Counsel_Handout_BT'] == 1) |
                               (full['Counsel_Parent_BT'] == 1), 1, 0)

    full['strong_bt'] = np.where((full['Refer_Parent_BT'] == 1) |
                                 (full['Refer_School_BT'] == 1), 1, 0)

#     full['bt_yn'] = np.where((full['weak_bt'] == 1) |
#                              (full['strong_bt'] == 1), 1, 0)

    return full

In [ ]:
# get required structured data for model in dict
## Structure
## tabular_data = pd.DataFrame({
##     'dis_symp': [0, 1, 1], # 0 = disorder-level code; 1 = symptom-level code
##     'age_35_6': [1, 0, 1], # 0 = 3-5 years old; 1 = 6 years old
## })
# the required variables are hard-coded into the function - make sure to change if needed
# patient ID is set as ANON_ID

def get_tabular_data(sdata, X_set):
    tab = pd.merge(sdata, X_set, left_on='ANON_ID', right_on=X_set.index, how='right')
    tab['dis_symp'] = tab['only.symp']
    tab['age_35_6'] = tab['ADHD.age'].apply(lambda x: 0 if x < 6 else 1)
    tab = tab.set_index('ANON_ID')
    tab = tab[['dis_symp', 'age_35_6']].to_dict('records')
    
    return tab

In [ ]:
# construct dataset with note text, structured data, and note labels
def get_dataset(text_data, tabular_data, labels):
    dataset = [
        {
            'input_ids': text_data['input_ids'][idx],
            'attention_mask': text_data['attention_mask'][idx],
            'tabular_data': torch.tensor(list(sample.values()), dtype=torch.float32),
            'labels': torch.tensor(labels[idx], dtype=torch.long)
        }
        for idx, sample in enumerate(tabular_data)
    ]
    return dataset

### Evaluation metrics

In [ ]:
# set metrics for comparison
def compute_metrics(p):
    preds = p.predictions.argmax(axis=1)
    probabilities = torch.sigmoid(torch.tensor(p.predictions)).cpu().numpy()
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    roc_auc = roc_auc_score(labels, probabilities[:, 1])

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
    }

In [ ]:
# get precision, recall, and f1-score at each threshold --> output as dataframe
def precision_recall_metrics(true_label , pred_prob):
    precision, recall, thresholds = sklearn.metrics.precision_recall_curve(true_label, pred_prob)
    precision = precision[:-1]
    recall = recall[:-1]
    f1 = 2*(precision*recall)/(precision+recall)
    results_DF = pd.DataFrame(data = {'precision': precision, 
                                       'recall': recall,
                                       'f1' : f1,
                                      'thresholds':thresholds})
    print(results_DF)
    return results_DF

In [ ]:
# produce confusion matrix after threshold selection
def confusion_matrix_thr(threshold_final, true_label, pred_prob):
    
    pred_label = [0 if x < threshold_final else 1 for x in pred_prob]

    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(true_label, pred_label, normalize='true').ravel()
    print(sklearn.metrics.classification_report(true_label, pred_label))

    print("tn:",tn)
    print("tp:",tp)
    print("fn:",fn)
    print("fp:",fp)
    
    return pred_label

In [ ]:
# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count, sample_weight):
    if sample_weight is None:
        return fastDeLong_no_weights(predictions_sorted_transposed, label_1_count)
    else:
        return fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight)


def fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank_weight(positive_examples[r, :], sample_weight[:m])
        ty[r, :] = compute_midrank_weight(negative_examples[r, :], sample_weight[m:])
        tz[r, :] = compute_midrank_weight(predictions_sorted_transposed[r, :], sample_weight)
    total_positive_weights = sample_weight[:m].sum()
    total_negative_weights = sample_weight[m:].sum()
    pair_weights = np.dot(sample_weight[:m, np.newaxis], sample_weight[np.newaxis, m:])
    total_pair_weights = pair_weights.sum()
    aucs = (sample_weight[:m]*(tz[:, :m] - tx)).sum(axis=1) / total_pair_weights
    v01 = (tz[:, :m] - tx[:, :]) / total_negative_weights
    v10 = 1. - (tz[:, m:] - ty[:, :]) / total_positive_weights
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def fastDeLong_no_weights(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating
              Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth, sample_weight):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions, sample_weight=None):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov

def get_auc_ci(y_pred, y_true):
    alpha = .95

    print(y_true.sum())
    auc, auc_cov = delong_roc_variance(
        y_true,
        y_pred)

    auc_std = np.sqrt(auc_cov)
    lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

    ci = stats.norm.ppf(
        lower_upper_q,
        loc=auc,
        scale=auc_std)

    ci[ci > 1] = 1

    print('AUC:', auc)
    print('AUC COV:', auc_cov)
    print('95% AUC CI:', ci)
    
    return auc, auc_cov, ci

# Import Data

In [ ]:
label_of_interest = "BT_yn"

In [ ]:
# pull and process original text data (takes in a directory of progress note txt files)
originalTextData = extractOriginalText2("cohort_2to6/Text files/combined_text")
print(originalTextData.shape)

In [ ]:
# pull and process annotations (takes in a directory of CLAMP outputted annotation XMI files)
annotatedXMIs = extractXMIAnnotation2("cohort_2to6/XMI files/combined")
print(annotatedXMIs.shape)

In [ ]:
# set binary outcome variable

annotatedXMIs['BT_yn'] = np.where((annotatedXMIs['Counsel_Parent_BT'] == 1) | (annotatedXMIs['Counsel_Handout_BT'] == 1) | (annotatedXMIs['Refer_Parent_BT'] == 1) | (annotatedXMIs['Refer_School_BT'] == 1), 1, 0)
annotatedXMIs['BT_yn'].value_counts()

In [ ]:
# merging data from both files 
data = originalTextData.merge(annotatedXMIs, on = "file", how = "right")

In [ ]:
# using imported function sectionize() from process_text for processing notes text data

data['extractText'] = data['note_des'].apply(lambda x: process_text.sectionize(x)[1])

In [ ]:
# using imported function clean_text() for processing notes text data

data['extractText'] = data['extractText'].apply(lambda x: process_text.clean_text(x))

In [ ]:
# read in structured data (not included in repository due to PHI)
structured_data = pd.read_csv("bt_demographics.csv")
# structured_data.head(1)

In [ ]:
# set patient ID type to int
data.anon_id = data.anon_id.astype(int)

In [ ]:
# merge text data with structured data to get complete dataset
data = pd.merge(structured_data, data, left_on='ANON_ID', right_on='anon_id', how='right')

In [ ]:
# filter down columns in dataset to those necessary for analysis:
## text and label
data = data.loc[:, ['extractText',label_of_interest, 'ANON_ID']]\
       .rename(columns = {'extractText':'text',
                          label_of_interest: 'label'})

data = data.set_index('ANON_ID')

In [ ]:
X = data.loc[:, 'text']
y = data.loc[:, 'label']

# Split the Data

In [ ]:
# split data into train, validation, and test sets (stratified)
## 70/30 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 117, stratify = y)
X_val_train, X_val_test, y_val_train, y_val_test = train_test_split(X_train, y_train, test_size = 0.3, random_state = 117, stratify = y_train)

In [ ]:
val_train = pd.concat([X_val_train, y_val_train], axis = 1)
val_test = pd.concat([X_val_test, y_val_test], axis = 1)
test = pd.concat([X_test, y_test], axis = 1)

 #Checking the final size for train, validation and test set

In [ ]:
print("X_train shape: ", X_val_train.shape)
print("X_val shape: ", X_val_test.shape)
print("X_test shape: ", X_test.shape)

In [ ]:
# outcome distribution in train set
y_train.value_counts()

In [ ]:
# outcome distribution in validation set
y_val.value_counts()

In [ ]:
# outcome distribution in test set
y_test.value_counts()

# Get Dataset for Model

In [ ]:
tokenizer = BertTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

In [ ]:
# reduce structured dataframe to only include the selected variables
red_sdata = structured_data[['ANON_ID', 'ever.dis', 'only.symp', 'ADHD.age']]

# train set tabular data
train_tab = get_tabular_data(red_sdata, X_val_train)
print(train_tab[0])

# validation set tabular data
val_tab = get_tabular_data(red_sdata, X_val_test)
print(val_tab[0])

# test set tabular data
test_tab = get_tabular_data(red_sdata, X_test)
print(test_tab[0])

In [ ]:
# Tokenize and get labels in correct format for train and validation sets
train_tok_text = tokenizer(list(X_val_train.values), return_tensors="pt", padding=True, truncation=True, max_length=512)
train_labels = list(y_val_train)

val_tok_text = tokenizer(list(X_val_test.values), return_tensors="pt", padding=True, truncation=True, max_length=512)
val_labels = list(y_val_test)

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")

    print('%d GPU(s) available' % torch.cuda.device_count())
    print('GPU name: ', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using CPU')
    device = torch.device("cpu")

In [ ]:
# get train and validation datasets
train_dataset = get_dataset(train_tok_text, train_tab, train_labels)
valid_dataset = get_dataset(val_tok_text, val_tab, val_labels)

# Run loaded model

In [ ]:
# Load pre-trained BERT model from checkpoint
bert_model = BertForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', num_labels=2)

checkpoint = torch.load('./bioclinicalbert_tabular_model_tuned/pytorch_model.bin')
args = torch.load('./bioclinicalbert_tabular_model_tuned/training_args.bin')
bert_model.load_state_dict(checkpoint)

bert_model = bert_model.to('cuda')

In [ ]:
# set trainer with model and args
trainer = Trainer(
    model=bert_model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# get performance on validation set (should be same as above)
val_pred_output = trainer.predict(valid_dataset)
val_eval_metrics = trainer.evaluate()
val_eval_metrics

In [ ]:
### output validation performance to file ###
# sourceFile = open('./bioclinicalbert_tabular_model_tuned/val_eval_metrics.txt', 'w')
# print(val_eval_metrics, file = sourceFile)
# sourceFile.close()

In [ ]:
# get validation probabilities and labels
val_probabilities = val_pred_output.predictions
val_probabilities = [x[1] for x in np.array([softmax(element) for element in val_probabilities])]
val_true_label = val_pred_output.label_ids

# put results in dataframe and save as csv
val_results = pd.DataFrame()
val_results['val_probabilities'] = val_probabilities
val_results['val_true_label'] = val_true_label
display(val_results.head(1))
# val_results.to_csv('./bioclinicalbert_tabular_model_tuned/val_results.csv')

In [ ]:
# get precision, recall, and f1-score at each threshold
res_df = precision_recall_metrics(val_true_label, val_probabilities)

In [ ]:
# get precision recall curve for validation set

p, r, t = precision_recall_curve(val_true_label, val_probabilities)
auprc = auc(r, p)
print("AUPRC: ", auprc)

plt.figure(dpi=1200) # dpi=1200
plt.plot(r, p, 'k', label='BioClinicalBERT (AUC = %s)' % str(auprc.round(2))[0:4])
plt.ylabel("Precision")
plt.xlabel("Recall")
# plt.title("Precision-Recall curve")
plt.legend(loc='lower left')
# plt.savefig('bioclinical_bert_pr_curve.jpg', dpi=1200)

In [ ]:
# Tokenize and get labels in correct format for the test set

test_tok_text = tokenizer(list(X_test.values), return_tensors="pt", padding=True, truncation=True, max_length=512)
test_labels = list(y_test)

test_dataset = get_dataset(test_tok_text, test_tab, test_labels)

In [ ]:
# get results on the test set without using the threshold

test_results = trainer.evaluate(test_dataset)
test_pred_output = trainer.predict(test_dataset)
test_results

In [ ]:
#### save test set results ####
# sourceFile = open('./bioclinicalbert_tabular_model_tuned/test_eval_metrics.txt', 'w')
# print(test_results, file = sourceFile)
# sourceFile.close()

In [ ]:
# threshold selection was done on the validation set

threshold = 0.001152

In [ ]:
# get test set results using the threshold

test_probabilities = test_pred_output.predictions
test_probabilities = [x[1] for x in np.array([softmax(element) for element in test_probabilities])]
test_predictions = [0 if x < threshold else 1 for x in test_probabilities]
test_true_label = test_pred_output.label_ids
print(test_true_label[0], test_predictions[0])

# put results in dataframe and save as csv
test_results = pd.DataFrame()
test_results['test_probabilities'] = test_probabilities
test_results['test_true_label'] = test_true_label
display(test_results.head(1))
# test_results.to_csv('./bioclinicalbert_tabular_model_tuned/test_results.csv')

In [ ]:
# get confusion matrix for test set after threshold selection
test_arr = confusion_matrix_thr(threshold,test_true_label,test_probabilities)

In [ ]:
# get AUROC with confidence intervals for model performance on test set
auc1, auc_cov1, ci1 = get_auc_ci(pd.Series(test_predictions), pd.Series(test_true_label))